In [85]:
#1-1
#read data
import pandas as pd

df = pd.read_csv('D1.csv')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 6 columns):
patient_id    1509 non-null int64
global_num    959 non-null float64
date          1509 non-null object
location      1509 non-null object
latitude      1509 non-null float64
longitude     1509 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 70.8+ KB
None


In [86]:
#1-2

#Group by patient_id
transactions = df.groupby(['patient_id'])['location'].apply(list)

print(transactions.head(10))


patient_id
1000000001                [Gyeonggi-do_Gimpo-si, Seoul_Jung-gu]
1000000002    [Seoul_Seongdong-gu, Seoul_Dongdaemun-gu, Seou...
1000000004                                  [Seoul_Jungnang-gu]
1000000005                                  [Seoul_Jungnang-gu]
1000000006                              [Gyeonggi-do_Goyang-si]
1000000007                              [Gyeonggi-do_Goyang-si]
1000000008                                      [Seoul_Jung-gu]
1000000009                                  [Seoul_Jungnang-gu]
1000000010                                  [Seoul_Seongbuk-gu]
1000000011                       [Seoul_Jung-gu, Seoul_Mapo-gu]
Name: location, dtype: object


In [87]:
from apyori import apriori

#transactions to normal list format
transaction_list = list(transactions)
results = list(apriori(transaction_list,min_support=0.01,min_confidence=0.01))

print(results[:5])

[RelationRecord(items=frozenset({'Busan_Busanjin-gu'}), support=0.012345679012345678, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Busanjin-gu'}), confidence=0.012345679012345678, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Dong-gu'}), support=0.01122334455667789, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Dong-gu'}), confidence=0.01122334455667789, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Dongnae-gu'}), support=0.019079685746352413, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Dongnae-gu'}), confidence=0.019079685746352413, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Haeundae-gu'}), support=0.015712682379349047, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Haeundae-gu'}), confidence=0.015712682379349047, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Seo-gu'}), support=0.0168350

In [88]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results : 
        for rule in rule_set.ordered_statistics :
            rules.append([','.join(rule.items_base),','.join(rule.items_add),
                        rule_set.support,rule.confidence,rule.lift])
            
    return pd.DataFrame(rules,columns = ['Left_side','Right_side','Support',
                                        'Confidence','Lift'])

result_df = convert_apriori_results_to_pandas_df(results)

result_df = result_df.sort_values(by = 'Lift',ascending=False)

print(result_df.head(10))


             Left_side          Right_side   Support  Confidence      Lift
49        Daegu_Buk-gu       Daegu_Jung-gu  0.011223    0.294118  5.348139
50       Daegu_Jung-gu        Daegu_Buk-gu  0.011223    0.204082  5.348139
56    Seoul_Gangnam-gu     Incheon_Jung-gu  0.028058    0.500000  3.349624
55     Incheon_Jung-gu    Seoul_Gangnam-gu  0.028058    0.187970  3.349624
59   Seoul_Jungnang-gu     Incheon_Jung-gu  0.019080    0.215190  1.441610
58     Incheon_Jung-gu   Seoul_Jungnang-gu  0.019080    0.127820  1.441610
62  Seoul_Yangcheon-gu     Incheon_Jung-gu  0.012346    0.203704  1.364662
61     Incheon_Jung-gu  Seoul_Yangcheon-gu  0.012346    0.082707  1.364662
40                      Seoul_Seodaemun-gu  0.016835    0.016835  1.000000
39                         Seoul_Seocho-gu  0.026936    0.026936  1.000000


In [89]:
#1-3

#list most of possible result 
results2 = list(apriori(transaction_list,min_support=0.001,min_confidence=0.001))
result_df2 = convert_apriori_results_to_pandas_df(results2)

#Exploit data that Left_side is Busan_Buk-gu
route = result_df2[result_df2['Left_side'] == "Busan_Buk-gu"]

route_sorted = route.sort_values(by = "Support",ascending = False)

print(route_sorted.head(5))

         Left_side                        Right_side   Support  Confidence  \
158   Busan_Buk-gu                   Busan_Yeonje-gu  0.003367         0.6   
155   Busan_Buk-gu                  Busan_Gangseo-gu  0.003367         0.6   
161   Busan_Buk-gu                    Gwangju_Buk-gu  0.002245         0.4   
1868  Busan_Buk-gu  Busan_Yeonje-gu,Busan_Gangseo-gu  0.002245         0.4   
1910  Busan_Buk-gu    Busan_Yeonje-gu,Gwangju_Buk-gu  0.002245         0.4   

            Lift  
158    10.482353  
155    76.371429  
161   118.800000  
1868  118.800000  
1910  118.800000  


In [90]:
#1-4
from collections import defaultdict
import re

def create_input_file(sequences):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            output_dict[str(item_id)] = itemset   ## Extra
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
            
    return output_dict

def convert_output_file(sequences, output_dict):
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
          left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
          sup = int(sup) / len(sequences)
          conf = float(conf)
          output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])

    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])